# 1. Importing modules and functions

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import chembl_structure_pipeline
from molvs import standardize_smiles
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from rdkit.Chem import MACCSkeys

[14:05:32] Initializing Normalizer


# 2.Data entry and curation work set

In [2]:
uploaded_file_ws="datasets/HDAC6_work.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  3083 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [3]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [4]:
len(y_tr)

3083

# 3.Standardization SDF file for work set

In [5]:
records_ws = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToMolBlock(all_mols_ws[i])
    records_ws.append(record)
            
mols_ws = []
for i,record in enumerate(records_ws):
    standard_record = chembl_structure_pipeline.standardize_molblock(record)
    m = Chem.MolFromMolBlock(standard_record)
    mols_ws.append(m)
           
moldf_ws = []
for val in mols_ws:
    if val != None:
        moldf_ws.append(val)
print('Kept data: ', len(moldf_ws), 'molecules')

[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharger
[14:05:34] Running Normalizer
[14:05:34] Running Uncharg

Kept data:  3083 molecules


[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharg

# 4.Data entry and curation test set

In [6]:
uploaded_file_ts="datasets/HDAC6_test.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  771 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [7]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [8]:
len(y_ts)

771

# 5.Standardization SDF file for test set

In [9]:
records_ts = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToMolBlock(all_mols_ts[i])
    records_ts.append(record)
            
mols_ts = []
for i,record in enumerate(records_ts):
    standard_record = chembl_structure_pipeline.standardize_molblock(record)
    m = Chem.MolFromMolBlock(standard_record)
    mols_ts.append(m)
           
moldf_ts = []
for val in mols_ts:
    if val != None:
        moldf_ts.append(val)
print('Kept data: ', len(moldf_ts), 'molecules')

[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharger
[14:05:39] Running Normalizer
[14:05:39] Running Uncharg

Kept data:  771 molecules


[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharger
[14:05:41] Running Normalizer
[14:05:41] Running Uncharg

## Calculation MACCS Fingerprints for work set

In [10]:
fp_tr = [MACCSkeys.GenMACCSKeys(m) for m in moldf_ws]

In [11]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [12]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [13]:
savetxt('Models/MACCS/x_tr_MACCS.csv', x_tr, delimiter=',')

In [14]:
x_tr.shape

(3083, 167)

## Calculation  MACCS Fingerprint for test set

In [15]:
fp_ts = [MACCSkeys.GenMACCSKeys(m) for m in moldf_ts]

In [16]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [17]:
x_ts = rdkit_numpy_convert(fp_ts)

In [18]:
x_ts.shape

(771, 167)

In [19]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

# 8. SVM model building and validation

In [22]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [23]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [24]:
svm = GridSearchCV(SVR(C=1.0, epsilon=0.2), param_grid, n_jobs=2, cv=cv, verbose=1)

In [25]:
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(epsilon=0.2), n_jobs=2,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [26]:
svm.best_params_
best_svm = svm.best_estimator_

In [27]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [30]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_svm), 2)
Q2_CV

0.6

In [31]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_svm)), 2)
RMSE_CV

0.72

# 9. Prediction for test set's molecules

In [32]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [33]:
y_pred_svm = best_svm.predict(x_ts)

In [34]:
Q2_TS = round(r2_score(y_ts, y_pred_svm), 2)
Q2_TS

0.6

In [35]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_svm)), 2)
RMSE_TS

0.72

save the model to disk

In [36]:
pickle.dump(best_svm, open('Models/MACCS/HDAC6_SVM_MACCS.pkl', 'wb'))

load the model from disk

In [89]:
best_svm = pickle.load(open('Models/MACCS/HDAC6_SVM_MACCS.pkl', 'rb'))

# 10. Y-randomization RF model

In [38]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_svm, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=seed)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   34.5s


True score =  0.6 
Y-randomization =  -0.39 
p-value =  0.0196


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   59.2s finished


# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [39]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [40]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,3073,3074,3075,3076,3077,3078,3079,3080,3081,3082
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.236068,4.123106,3.316625,1.414214,5.099020,2.449490,3.464102,2.236068,2.828427,3.872983,...,1.000000,1.414214,2.000000,2.000000,0.000000,1.414214,1.414214,2.828427,1.414214,2.449490
2,3.464102,4.242640,3.464102,1.732051,5.477226,3.872983,3.872983,2.449490,5.099020,4.582576,...,2.828427,2.828427,2.449490,2.000000,0.000000,2.828427,1.732051,3.741657,1.732051,2.449490
3,3.605551,4.472136,3.872983,1.732051,5.567764,3.872983,4.000000,2.449490,5.099020,4.582576,...,2.828427,3.000000,2.449490,2.449490,0.000000,3.000000,2.236068,3.872983,2.236068,2.645751
4,4.000000,4.472136,4.123106,1.732051,5.744563,4.690416,4.000000,2.645751,5.099020,4.690416,...,2.828427,3.162278,3.000000,2.449490,1.000000,3.000000,2.449490,4.242640,2.449490,2.645751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,8.062258,7.937254,8.000000,8.426149,9.380832,8.124039,8.124039,7.874008,8.831760,7.874008,...,7.937254,8.306623,8.306623,7.937254,8.000000,8.306623,8.246211,8.062258,8.062258,7.937254
3079,8.124039,8.000000,8.062258,8.426149,9.433981,8.124039,8.124039,7.874008,8.831760,7.874008,...,7.937254,8.366600,8.366600,7.937254,8.000000,8.306623,8.246211,8.062258,8.124039,7.937254
3080,8.185352,8.000000,8.124039,8.426149,9.539392,8.124039,8.185352,7.874008,8.831760,8.000000,...,8.000000,8.426149,8.602325,8.000000,8.124039,8.366600,8.246211,8.124039,8.124039,7.937254
3081,8.246211,8.000000,8.185352,8.944272,9.591663,8.185352,8.185352,7.937254,8.831760,8.426149,...,8.062258,8.426149,8.602325,8.000000,8.306623,8.544003,8.306623,8.185352,8.185352,8.000000


In [41]:
similarity= neighbors_k

In [42]:
Dmean=np.mean(similarity[1,:])

In [43]:
round(Dmean, 2)

1.36

In [44]:
std=np.std(similarity[1,:])

In [45]:
round(std, 2)

1.11

In [46]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

1.91


In [47]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [48]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,761,762,763,764,765,766,767,768,769,770
0,3.464102,3.872983,0.000000,3.605551,2.000000,0.000000,1.414214,1.732051,1.414214,1.414214,...,1.414214,2.000000,1.000000,1.414214,0.000000,3.162278,1.414214,2.000000,1.000000,2.236068
1,4.000000,4.123106,1.732051,4.123106,2.236068,2.828427,1.414214,2.236068,2.236068,1.732051,...,1.732051,2.000000,1.000000,1.732051,0.000000,3.741657,2.000000,2.000000,3.316625,2.449490
2,4.472136,4.123106,2.000000,4.358899,2.449490,5.099020,2.645751,2.236068,3.000000,1.732051,...,2.000000,2.000000,2.449490,1.732051,0.000000,3.741657,2.000000,2.000000,3.316625,2.645751
3,4.472136,4.358899,2.000000,4.358899,2.449490,5.099020,2.645751,2.236068,4.472136,2.828427,...,2.236068,2.236068,2.645751,2.000000,0.000000,3.872983,2.000000,2.000000,3.464102,2.828427
4,4.582576,4.582576,3.162278,4.472136,2.645751,5.385165,2.828427,2.449490,4.472136,2.828427,...,2.236068,2.449490,2.645751,2.828427,1.000000,3.872983,2.236068,2.000000,3.464102,2.828427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,8.306623,8.602325,8.426149,8.306623,8.185352,8.660254,7.681146,8.426149,8.306623,8.660254,...,8.426149,7.810250,8.000000,7.937254,8.062258,8.246211,8.366600,8.246211,7.937254,7.937254
3079,8.426149,8.660254,8.485281,8.306623,8.246211,8.660254,7.745967,8.426149,8.366600,8.660254,...,8.426149,7.937254,8.000000,8.000000,8.124039,8.306623,8.366600,8.306623,8.000000,7.937254
3080,8.485281,8.660254,8.485281,8.306623,8.246211,8.660254,7.810250,8.485281,8.426149,8.717798,...,8.426149,7.937254,8.000000,8.062258,8.124039,8.306623,8.426149,8.366600,8.000000,8.000000
3081,8.485281,8.660254,8.544003,8.366600,8.246211,8.660254,7.810250,8.544003,8.426149,8.717798,...,8.485281,7.937254,8.124039,8.062258,8.426149,8.426149,8.485281,8.426149,8.124039,8.000000


In [49]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[3.464 3.873 0.    3.606 2.    0.    1.414 1.732 1.414 1.414 1.    1.
 3.742 1.732 0.    0.    1.    2.    2.449 2.236 2.449 1.414 2.236 1.
 4.472 1.    0.    4.123 1.414 2.449 1.732 2.646 3.317 3.742 1.732 1.414
 2.449 1.732 1.732 1.732 2.646 1.414 2.236 2.646 2.646 2.236 1.414 1.732
 1.414 0.    2.449 1.732 3.606 3.    2.646 0.    1.732 1.414 2.449 1.414
 1.732 1.732 2.449 0.    1.732 4.    2.    1.414 0.    2.449 1.    1.414
 0.    2.646 1.414 1.    1.732 2.449 3.162 1.414 0.    2.236 1.732 1.732
 3.606 1.414 0.    1.    3.606 0.    0.    2.449 1.732 1.732 0.    3.873
 2.    0.    0.    2.646 0.    3.742 0.    1.414 2.449 2.646 1.414 2.236
 2.    1.    1.    1.732 2.449 0.    2.    1.414 2.    0.    2.449 0.
 2.    0.    0.    2.828 2.828 2.828 1.    1.    2.828 0.    1.414 1.
 1.732 1.414 3.606 1.    2.828 1.732 0.    0.    2.236 1.732 1.414 0.
 2.    2.    0.    1.    2.236 1.    2.828 1.414 2.    3.606 2.449 1.
 0.    2.    1.732 0.    1.414 1.414 2.    0.    0.    0.    1.414 3.

In [50]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[False False  True False False  True  True  True  True  True  True  True
 False  True  True  True  True False False False False  True False  True
 False  True  True False  True False  True False False False  True  True
 False  True  True  True False  True False False False False  True  True
  True  True False  True False False False  True  True  True False  True
  True  True False  True  True False False  True  True False  True  True
  True False  True  True  True False False  True  True False  True  True
 False  True  True  True False  True  True False  True  True  True False
 False  True  True False  True False  True  True False False  True False
 False  True  True  True False  True False  True False  True False  True
 False  True  True False False False  True  True False  True  True  True
  True  True False  True False  True  True  True False  True  True  True
 False False  True  True False  True False  True False False False  True
  True False  True  True  True  True False  True  T

In [64]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.67


In [52]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  2   5   6   7   8   9  10  11  13  14  15  16  21  23  25  26  28  30
  34  35  37  38  39  41  46  47  48  49  51  55  56  57  59  60  61  63
  64  67  68  70  71  72  74  75  76  79  80  82  83  85  86  87  89  90
  92  93  94  97  98 100 102 103 106 109 110 111 113 115 117 119 121 122
 126 127 129 130 131 132 133 135 137 138 139 141 142 143 146 147 149 151
 155 156 158 159 160 161 163 164 165 166 168 169 170 172 174 175 176 180
 181 184 185 187 190 192 193 195 196 199 200 201 202 203 204 205 206 208
 209 210 211 212 214 219 220 221 222 223 224 225 226 230 231 232 233 234
 235 237 238 239 240 241 242 245 246 248 249 250 251 253 254 255 256 257
 261 262 264 265 266 270 271 273 275 277 278 279 280 281 282 284 286 288
 289 290 292 294 295 296 297 299 300 302 304 305 306 308 309 310 312 313
 314 315 316 317 318 320 321 324 325 327 332 334 335 336 337 340 342 344
 345 346 347 348 349 350 351 353 354 356 359 360 364 366 367 368 369 370
 371 372 37

In [53]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [54]:
y_pred_svm_ad=list(y_pred_svm)

In [55]:
y_pred_svm_ad[:] = [x for i,x in enumerate(y_pred_svm_ad) if i not in out_Ad]

In [56]:
len(y_pred_svm_ad)

517

In [57]:
y_ts_ad=list(y_ts)

In [58]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [59]:
len(y_ts_ad)

517

In [60]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_svm_ad), 2)
Q2_TS

0.64

In [61]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_svm_ad)), 2)
RMSE_TS

0.68